In [6]:
# Importa bibliotecas necessárias
import cv2  # OpenCV para processamento de imagens e vídeo
import mediapipe as mp  # MediaPipe para detecção de mãos
import json  # Para manipulação de arquivos JSON
import os  # Para lidar com arquivos e diretórios
import numpy as np  # Biblioteca matemática para cálculos vetoriais
import time  # Para medições de tempo e temporizadores

# Inicialização do MediaPipe para detecção de mãos
mp_hands = mp.solutions.hands  # Solução de detecção de mãos do MediaPipe
hands = mp_hands.Hands(
    static_image_mode=False,  # Define se usará imagens estáticas (False para vídeo)
    max_num_hands=1,  # Número máximo de mãos a detectar
    min_detection_confidence=0.7  # Confiança mínima para detectar mãos
)
mp_drawing = mp.solutions.drawing_utils  # Utilitário para desenhar conexões da mão

# Configuração do banco de dados
database_file = "hand_gestures.json"  # Nome do arquivo JSON com os dados dos gestos
current_phrase = ""  # Frase atualmente formada
letter_start_time = None  # Momento em que uma letra começa a ser analisada
pending_letter = None  # Letra que está sendo avaliada

# Coordenadas do retângulo onde a mão deve ser posicionada
RECT_TOP_LEFT = (200, 100)  # Canto superior esquerdo do retângulo
RECT_BOTTOM_RIGHT = (440, 400)  # Canto inferior direito do retângulo

# Função para carregar o banco de dados de gestos
def load_database():
    # Verifica se o arquivo de banco de dados existe
    if not os.path.exists(database_file):
        print("Banco de dados não encontrado.")  # Exibe mensagem de erro
        return {}  # Retorna um dicionário vazio
    with open(database_file, "r") as f:  # Abre o arquivo JSON em modo leitura
        return json.load(f)  # Retorna os dados carregados como um dicionário

# Função para calcular a distância média entre dois conjuntos de coordenadas
def calculate_mean_distance(input_coordinates, reference_coordinates):
    distances = [
        # Calcula a distância euclidiana entre cada par de coordenadas
        np.linalg.norm(np.array([ic[0], ic[1]]) - np.array([rc["x"], rc["y"]]))
        for ic, rc in zip(input_coordinates, reference_coordinates)
    ]
    return np.mean(distances)  # Retorna a distância média

# Função para reconhecer uma letra com base nas coordenadas da mão
def recognize_letter(input_coordinates, database):
    min_distance = float("inf")  # Define a menor distância inicial como infinito
    recognized_letter = None  # Inicializa a letra reconhecida como None

    # Itera sobre cada letra e suas variações no banco de dados
    for letter, variations in database.items():
        for variation in variations:
            # Calcula a distância média para a variação atual
            distance = calculate_mean_distance(input_coordinates, variation)
            if distance < min_distance:  # Verifica se essa é a menor distância
                min_distance = distance
                recognized_letter = letter  # Atualiza a letra reconhecida
    
    return recognized_letter  # Retorna a letra com menor distância

# Função para verificar se a mão está dentro do retângulo definido
def is_hand_inside_rectangle(hand_coordinates):
    for point in hand_coordinates:  # Itera sobre as coordenadas da mão
        # Converte as coordenadas normalizadas em pixels
        x_pixel = int(point[0] * 640)  # Largura da tela
        y_pixel = int(point[1] * 480)  # Altura da tela
        # Verifica se o ponto está dentro do retângulo
        if not (RECT_TOP_LEFT[0] <= x_pixel <= RECT_BOTTOM_RIGHT[0] and RECT_TOP_LEFT[1] <= y_pixel <= RECT_BOTTOM_RIGHT[1]):
            return False  # Retorna False se algum ponto estiver fora
    return True  # Retorna True se todos os pontos estiverem dentro

# Função principal para capturar os gestos da mão em tempo real
def capture_gestures(database):
    global current_phrase, letter_start_time, pending_letter  # Variáveis globais para manipulação
    cap = cv2.VideoCapture(0)  # Inicializa a captura de vídeo (câmera)

    while cap.isOpened():  # Enquanto a câmera estiver aberta
        success, image = cap.read()  # Lê um quadro da câmera
        if not success:  # Verifica se a captura foi bem-sucedida
            print("Erro ao acessar a câmera.")
            break

        # Converte o quadro para RGB (necessário para o MediaPipe)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)  # Processa o quadro para detectar mãos

        # Desenha um retângulo transparente na tela
        overlay = image.copy()
        cv2.rectangle(overlay, RECT_TOP_LEFT, RECT_BOTTOM_RIGHT, (0, 0, 255), -1)  # Preenchimento
        cv2.addWeighted(overlay, 0.2, image, 0.8, 0, image)  # Transparência
        cv2.rectangle(image, RECT_TOP_LEFT, RECT_BOTTOM_RIGHT, (0, 0, 255), 2)  # Contorno

        if results.multi_hand_landmarks:  # Se mãos forem detectadas
            for hand_landmarks in results.multi_hand_landmarks:
                # Desenha os pontos e conexões da mão detectada
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                # Converte as coordenadas normalizadas para uma lista
                hand_coordinates = [(landmark.x, landmark.y) for landmark in hand_landmarks.landmark]
                
                # Verifica se a mão está dentro do retângulo
                if is_hand_inside_rectangle(hand_coordinates):
                    letter = recognize_letter(hand_coordinates, database)  # Reconhece a letra
                    if letter:  # Se uma letra foi reconhecida
                        if pending_letter != letter:  # Atualiza a letra pendente
                            pending_letter = letter
                            letter_start_time = time.time()  # Marca o início do tempo
                        elapsed_time = time.time() - letter_start_time
                        if elapsed_time >= 2:  # Espera 2 segundos para confirmar a letra
                            cv2.putText(image, f"Pressione 'Enter' para confirmar: {pending_letter}",
                                        (10, 450), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)
                    else:  # Nenhuma letra reconhecida
                        pending_letter = None
                        letter_start_time = None
                else:  # Mão fora do retângulo
                    cv2.putText(
                        image,
                        "Posicione a mão dentro do retângulo!",
                        (10, 420),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.6,
                        (0, 0, 255),
                        2,
                        cv2.LINE_AA,
                    )

        # Exibe a frase e a fila de letras na tela
        if pending_letter:
            cv2.putText(image, f"Fila: {pending_letter}", 
                        (10, 100), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.6, 
                        (0, 0, 255), 
                        2, 
                        cv2.LINE_AA)
        cv2.putText(image, current_phrase, 
                    (10, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.7,
                    (0, 0, 0), 
                    1, 
                    cv2.LINE_AA)

        cv2.imshow("Reconhecimento de Gestos", image)  # Exibe o quadro na janela

        # Controle do teclado para interação
        key = cv2.waitKey(1) & 0xFF
        if key == ord('\r'):  # Enter para confirmar a letra pendente
            if pending_letter:
                current_phrase += pending_letter
                pending_letter = None
                letter_start_time = None
        elif key == ord(' '):  # Espaço para adicionar um espaço na frase
            current_phrase += " "
        elif key == ord('w'):  # AltGr + W para adicionar "?"
            current_phrase += "?"
        elif key == ord('!'):  # Shift + 1 para adicionar "!"
            current_phrase += "!"
        elif key == ord('q'):  # Q para sair do programa
            break

    cap.release()  # Libera a câmera
    cv2.destroyAllWindows()  # Fecha as janelas abertas

# Função principal que carrega o banco de dados e inicia o reconhecimento de gestos
def main():
    print("Carregando o banco de dados...")
    database = load_database()  # Carrega os dados do banco
    if not database:  # Verifica se o banco está vazio
        print("Nenhum dado disponível no banco. Por favor, cadastre letras primeiro.")
        return

    print("Iniciando o reconhecimento de gestos...")
    capture_gestures(database)  # Chama a função de captura de gestos

# Inicia o programa
main()


Carregando o banco de dados...
Iniciando o reconhecimento de gestos...
